In [57]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import os
import networkit as nk
import datetime
import pandas as pd

In [69]:
#graph_name = 'facebook100/Caltech36.mat'
name = 'karate'
output_path = '../output/QTM/' + name + '/'
graph_names = ['karate.graph']
initializations = ['trivial', 'editing', 'id_insert', 'ascending_degree_insert']
maxIterations = [0, 2, 5, 20]
sortPaths = [True, False]
randomness = [True, False]
plateauSize = [2, 5, 10]
seed = 0

In [59]:
#name = 'biological'
#output_path = '../output/QTM/' + name + '/'
#bio_dir = "../input/biological/graphs/"
#graph_names = []
#for filename in os.listdir(bio_dir):
#    graph_names.append("biological/graphs/" + filename)
#initializations = ['ascending_degree_insert']
#maxIterations = [10]
#sortPaths = [True]
#randomness = [False]
#plateauSize = [0]
#seed = 0

In [76]:
def testrun (G, graph_name, init, m, s, r, p):
    if(init == 'trivial'):
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [], p)
    if(init == 'editing'):
        G.indexEdges()
        editing = nk.community.QuasiThresholdEditingLinear(G)
        editing.run()
        mover = nk.community.QuasiThresholdEditingLocalMover(G, editing.getParents(), m, s, r, [], p)
    if(init == 'id_insert'):
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [i for i in range(G.upperNodeIdBound())], p)        
    if(init == 'ascending_degree_insert'):
        degree_dict = {}
        for u in G.iterNodes():
            degree_dict[u] = G.degree(u)
        mover = nk.community.QuasiThresholdEditingLocalMover(G, [], m, s, r, [el[0] for el in sorted(degree_dict.items(), key=lambda kv: kv[1])], p)
        
    a = datetime.datetime.now()  
    mover.run()   
    delta = datetime.datetime.now() - a
    if(r):
        return [graph_name, init, m, s, r, p, mover.getNumberOfEdits(), 
                mover.getUsedIterations(), mover.getPlateauSize(), delta.total_seconds() * 1000]
    else:
        return [graph_name, init, m, s, r, '-', mover.getNumberOfEdits(), 
                mover.getUsedIterations(), '-', delta.total_seconds() * 1000]



In [78]:
file_name = name + '_'
file_name += str(seed)
for init in initializations:
    file_name += init[0]
for el in maxIterations:
    file_name += str(el)
for el in sortPaths:
    if(el):
        file_name += '1'
    else:
        file_name += '0'
for el in randomness:
    if(el):
        file_name += '1'
    else:
        file_name += '0'
for el in plateauSize:
    file_name += str(el)
file_name += '.csv'

input_path = "../input/"

df = pd.DataFrame(columns  = ['graph', 'initialization', 'maxIterations', 'sortPaths', 'randomness', 'plateauSize','edits', 'usedIterations', 'actualPlateau', 'time'])

nk.setSeed(seed, False)
i = 0
for graph_name in graph_names:
    graph_path = input_path + graph_name
    if(graph_name.split('/')[0] ==  "facebook100"):
        G = nk.graphio.readMat(input_path + graph_name, key="A")
    if(graph_name.split('.')[1] == "graph"):
        G = nk.readGraph(graph_path, nk.Format.METIS)
    if(graph_name.split('.')[1] == "edgelist"):
        G = nk.readGraph(graph_path, nk.Format.EdgeListTabOne)

    for init in initializations:
        for m in maxIterations:
            for s in sortPaths:
                for r in randomness:
                    if(r):
                        for p in plateauSize:
                            df.loc[i] = testrun(G, graph_name, init, m, s, r, p)
                            i += 1
                    else:
                        df.loc[i] = testrun(G, graph_name, init, m, s, r, 0)
                        i+=1
                    

result_path = output_path + file_name
if not os.path.exists(output_path):
    os.makedirs(output_path)
df.to_csv(result_path, sep=',', encoding='utf-8')
                   







In [68]:
df = pd.read_csv('../output/QTM/biological/bio_data.csv')
new_df = pd.DataFrame(columns = ['Graph', 'Solved', 'k', 'QTM k', 'ascending_degrees/sortPaths/0', 'ascending_degrees/sortPaths/10'])
for index, row in df.iterrows():
    G = nk.readGraph("../input/biological/graphs/" + row['Graph'].split('.')[0] + ".graph", nk.Format.METIS)
    degree_dict = {}
    for u in G.iterNodes():
        degree_dict[u] = G.degree(u)
    mover = nk.community.QuasiThresholdEditingLocalMover (G, [], 0, True, False, [el[0] for el in sorted(degree_dict.items(), key=lambda kv: kv[1])], 0)
    mover.run()
    mover2 = nk.community.QuasiThresholdEditingLocalMover (G, [], 10, True, False, [el[0] for el in sorted(degree_dict.items(), key=lambda kv: kv[1])], 0)
    mover2.run()
    new_df.loc[index] = [row['Graph'], row['Solved'], row['k'], row['QTM k'], mover.getNumberOfEdits(), mover2.getNumberOfEdits()]
      
    
print(new_df)
new_df.to_csv('../output/QTM/biological/bio_data_extended.csv', sep=',', encoding='utf-8')

                                                  Graph Solved    k  QTM k  \
0     cost_matrix_component_nr_1000_size_31_cutoff_10.0   True    0      0   
1     cost_matrix_component_nr_1001_size_19_cutoff_10.0   True    5      5   
2     cost_matrix_component_nr_1002_size_33_cutoff_10.0   True    2      2   
3     cost_matrix_component_nr_1003_size_62_cutoff_10.0   True   46     46   
4     cost_matrix_component_nr_1004_size_106_cutoff_...   True  163    163   
5     cost_matrix_component_nr_1005_size_64_cutoff_10.0   True  114    116   
6     cost_matrix_component_nr_1006_size_43_cutoff_10.0   True   32     32   
7     cost_matrix_component_nr_1007_size_53_cutoff_10.0   True   63     64   
8     cost_matrix_component_nr_1008_size_10_cutoff_10.0   True    2      2   
9     cost_matrix_component_nr_1009_size_28_cutoff_10.0   True   41     41   
10     cost_matrix_component_nr_100_size_49_cutoff_10.0  False  147    154   
11    cost_matrix_component_nr_1010_size_108_cutoff_...   True  